In [62]:
import pandas as pd
import json
import csv
import re
import tweepy
from tweepy import OAuthHandler
import time

In [63]:
df_archive = pd.read_csv('twitter-archive-enhanced-2.csv')
df_image = pd.read_csv('image-predictions-3.tsv',  delimiter='\t')

In [48]:
df_archive.head()
#df.info()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [25]:
consumer_key = 'jbL7N3K4eXgD1zhDfeS2TcnFr'
consumer_secret = 'XeIpJqZ4i8WAolHVCNBqGtILZKY1J0TVJw8GkpW9jDRthls1qn'
access_token = '495295426-3dc2rEDWSI8AX8NxYCpIRV9tk83EwocP7RzbraDC'
access_token_secret = 'zsulnNzoCcHRURBOkfVkb1tLcJPfYinSwfJBpn9Nam1FV'


# search_favorites(consumer_key, consumer_secret, access_token, access_token_secret):
#create authentication for accessing twitter
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#initialize Tweepy API
api = tweepy.API(auth, wait_on_rate_limit =True, wait_on_rate_limit_notify = True)
tweet_ids = df_archive.tweet_id



Rate limit reached. Sleeping for: 611
Rate limit reached. Sleeping for: 593


In [114]:
count = 0
fails_dict = {}
#save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    
    #loop through each tweet id and write to outfile
    for tweet_id in tweet_ids:
        count += 1        
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, outfile)
            outfile.write('\n')
            
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass


Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail


Rate limit reached. Sleeping for: 595


Fail
Fail


Rate limit reached. Sleeping for: 609


In [116]:
df_1 = pd.DataFrame(columns=['tweet_id', 'retweet_count', 'favorite_count'])
with open('tweet_json.txt') as f:
    for line in f:
        status  = json.loads(line)
        tweet_id = status['id_str']
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
        df_1 = df_1.append(pd.DataFrame([[tweet_id, retweet_count, favorite_count]],
                                        columns=['tweet_id', 'retweet_count', 'favorite_count']))
df_1 = df_1.reset_index(drop=True)


,tweet_id,retweet_count,favorite_count
0,892420643555336193,8196,37562
1,892177421306343426,6060,32289
2,891815181378084864,4008,24334
3,891689557279858688,8352,40919
4,891327558926688256,9045,39111


In [316]:
df_1.tweet_id = df_1.tweet_id.astype('str')
df_archive.tweet_id = df_archive.tweet_id.astype('str')
#assess
df_merged2.query('rating_numerator < 10 or rating_numerator >16')['text']
df_merged2.name.value_counts()


None         603
a             55
Lucy          11
Cooper        10
Oliver        10
Charlie       10
Tucker         9
Penny          9
Winston        8
Lola           8
Sadie          8
the            8
Daisy          7
Toby           7
Bella          6
an             6
Bo             6
Bailey         6
Koda           6
Oscar          6
Jax            6
Stanley        6
Leo            5
Buddy          5
Milo           5
Louis          5
Rusty          5
Scout          5
Chester        5
Bentley        5
            ... 
Fiji           1
Craig          1
Cali           1
Mingus         1
Lilah          1
Kanu           1
Blue           1
Fabio          1
Vince          1
Sprinkles      1
Sunshine       1
Beckham        1
Fizz           1
Margo          1
Bloo           1
Marq           1
Thor           1
Rufio          1
Goliath        1
Alfy           1
Scott          1
Frönq          1
Mack           1
Rodman         1
Edgar          1
Sandra         1
Mary           1
Arya          

Merge data frames

In [317]:
df_merged = df_archive.merge(df_1, how= 'inner', on = 'tweet_id')
#df_archive.info()
#df_1.info()

def: remove rows present in retweet status
remove unnecessary rows
remove retweet status columns
remove denomenator

In [318]:
#df1 = df1[df1['retweeted_status_id'].isna()]
df_merged2 = df_merged[df_merged['retweeted_status_id'].isna()]
df_merged2 = df_merged[df_merged['in_reply_to_status_id'].isna()]

df_merged2.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'rating_denominator', 'in_reply_to_status_id', 'in_reply_to_user_id'],  axis =1, inplace = True )

combine stages

In [319]:
#df_merged2['stage'] = df_merged2.doggo + df_merged2.floofer + df_merged2.pupper + df_merged2.puppo
#df_merged2.doggo.value_counts()


remove rows without photos

In [522]:
#df_merged2.info()
#df_merged['image_link'] = df_merged.text.str.extract('http://')
df_merged3 = df_merged2[df_merged2.text.str.contains("https://")]

Names 869988702071779329 = quite
a 
an 
the

In [513]:
df_merged3.loc[df_merged3['name'] == "quite", 'name'] = None
df_merged3.loc[df_merged3['name'] == "the", 'name'] = None
df_merged3.loc[df_merged3['name'] == "an", 'name'] = None
df_merged3.loc[df_merged3['name'] == "an", 'name'] = None



combine stage columns

In [447]:

df_merged3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2227 entries, 0 to 2336
Data columns (total 13 columns):
tweet_id            2227 non-null object
timestamp           2227 non-null object
source              2227 non-null object
text                2227 non-null object
expanded_urls       2227 non-null object
rating_numerator    2227 non-null int64
name                2209 non-null object
doggo               2227 non-null object
floofer             2227 non-null object
pupper              2227 non-null object
puppo               2227 non-null object
retweet_count       2227 non-null object
favorite_count      2227 non-null object
dtypes: int64(1), object(12)
memory usage: 243.6+ KB


In [523]:
df_merged3['stage'] = df_merged3.text.str.findall(r'(doggo|floofer|pupper|puppo)')


C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0             []
1             []
2             []
3             []
4             []
5             []
6             []
7             []
8             []
9        [doggo]
10            []
11            []
12       [puppo]
13            []
14       [puppo]
15            []
16            []
17            []
18            []
19            []
20            []
21            []
22            []
23            []
24            []
25            []
26            []
27            []
28      [pupper]
30            []
          ...   
2307          []
2308          []
2309          []
2310          []
2311          []
2312          []
2313          []
2314          []
2315          []
2316          []
2317          []
2318          []
2319          []
2320          []
2321          []
2322          []
2323          []
2324          []
2325          []
2326          []
2327          []
2328          []
2329          []
2330          []
2331          []
2332          []
2333          []
2334          

In [530]:
df_merged3['stage'] = df_merged3['stage'].str.join(',')


C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [540]:


df4['stage'].replace('pupper,pupper', 'pupper', inplace =True) 
df4['stage'].replace('pupper,doggo,doggo', 'pupper,doggo', inplace = True) 
df4['stage'].replace('pupper,pupper,pupper', 'pupper', inplace = True) 

df4.stage.value_counts()

                1849
pupper           251
doggo             81
puppo             31
doggo,pupper       6
floofer            4
pupper,doggo       3
puppo,doggo        2
Name: stage, dtype: int64

In [541]:
df4 = df_merged3.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis =1)

In [347]:
import datetime as dt

df4.timestamp = pd.to_datetime(df4['timestamp'])

df4['day_of_week'] = df4['timestamp'].dt.dayofweek

TweepError: [{'code': 144, 'message': 'No status found with that ID.'}]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [542]:
df_1 = df_image.query('p1_dog == True')
df_2 = df_image.query('p1_dog != True & p2_dog == True')
df_3 = df_image.query('p1_dog != True & p2_dog != True & p3_dog == True')

df_1['breed'] = df_1['p1']
df_2['breed'] = df_2['p2']
df_3['breed'] = df_3['p3']
df_3.breed
len(df_1) + len(df_2) + len(df_3)
len(df_image)

C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

2075

In [543]:
image_clean = pd.concat([df_1, df_2, df_3], axis=0, join='outer', join_axes=None, ignore_index=True,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
image_clean = image_clean[['tweet_id','breed']]

In [546]:
#image['tweet_id']
image_clean = image_clean.astype(str, copy=True)
df4.merge(image_clean, how = 'left', on = 'tweet_id')
df4.info()
df4.tweet_id.duplicated().any()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2227 entries, 0 to 2336
Data columns (total 10 columns):
tweet_id            2227 non-null object
timestamp           2227 non-null object
source              2227 non-null object
text                2227 non-null object
expanded_urls       2227 non-null object
rating_numerator    2227 non-null int64
name                2227 non-null object
retweet_count       2227 non-null object
favorite_count      2227 non-null object
stage               2227 non-null object
dtypes: int64(1), object(9)
memory usage: 271.4+ KB


False

(1532, 12)

Test:

In [547]:
df_image.query('p1_dog != True & p2_dog != True & p2_dog != True').shape

(388, 12)

Re extract data to test its accuracy

In [548]:
df_test = pd.DataFrame(columns=['tweet_id', 'text'])
with open('tweet_json.txt') as f:
    for line in f:
        status  = json.loads(line)
        tweet_id = status['id_str']
        text = status['full_text']
        df_test = df_test.append(pd.DataFrame([[tweet_id, text]],
                                        columns=['tweet_id', 'text']))
df_test = df_test.reset_index(drop=True)


,,0
,match,
9,0,doggo
12,0,puppo
14,0,puppo
28,0,pupper
42,0,doggo
45,0,floofer
48,0,pupper
53,0,pupper
55,0,pupper


In [517]:
df_merged3['stage']= df_merged3['floofer'] + df_merged3['doggo'] + df_merged3['pupper'] + df_merged3['puppo']





C:\Users\Summer\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


ValueError: Location based indexing can only have [labels (MUST BE IN THE INDEX), slices of labels (BOTH endpoints included! Can be slices of integers if the index is integers), listlike of labels, boolean] types

Series([], Name: stage, dtype: int64)

In [ ]:
change variable types
tweetid should be a string
stage characters should be categories 

fix the names column
meet, this is, 

Delete the rating denomenator

images: extract type of dog and merge with bigger table
